<a href="https://colab.research.google.com/github/Jacob682/PSTPL/blob/main/user_cf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import torch
import torch.nn.functional as F

In [6]:
ratings = torch.tensor([
    [5, 3, 0, 0, 2],
    [4, 0, 0, 2, 3],
    [0, 0, 3, 4, 0],
    [2, 3, 0, 5, 0],
    [1, 0, 0, 4, 5]],
    dtype = torch.float32
)

In [20]:
# 计算用户间相似度
matrix1 = ratings.unsqueeze(1) #(5,1,5) 广播后，一个用户的评分，复制5次，和matrix2 5个不同的用户计算
matrix2 = ratings.unsqueeze(0) #(1,5,5)
user_similarity = F.cosine_similarity(matrix1,matrix2,dim=2)
user_similarity.fill_diagonal_(0)
user_similarity

tensor([[1.0000, 0.7832, 0.0000, 0.5000, 0.3755],
        [0.7832, 1.0000, 0.2971, 0.5422, 0.7736],
        [0.0000, 0.2971, 1.0000, 0.6489, 0.4938],
        [0.5000, 0.5422, 0.6489, 1.0000, 0.5507],
        [0.3755, 0.7736, 0.4938, 0.5507, 1.0000]])

In [21]:
ratings.unsqueeze(1)

tensor([[[5., 3., 0., 0., 2.]],

        [[4., 0., 0., 2., 3.]],

        [[0., 0., 3., 4., 0.]],

        [[2., 3., 0., 5., 0.]],

        [[1., 0., 0., 4., 5.]]])

In [22]:
matrix2

tensor([[[5., 3., 0., 0., 2.],
         [4., 0., 0., 2., 3.],
         [0., 0., 3., 4., 0.],
         [2., 3., 0., 5., 0.],
         [1., 0., 0., 4., 5.]]])

In [23]:
# 用相似度加权预测评分
def predict(user_id, item_id):
  sim_scores = user_similarity[user_id]
  user_rating = ratings[:,item_id]
  # 考虑已评分用户
  non_zero_ratings = user_rating != 0 # 取bool
  relevent_user = sim_scores[non_zero_ratings]
  relevent_rating = user_rating[non_zero_ratings]

  numerator = torch.dot(relevent_user,relevent_rating)
  denominator = relevent_user.sum()
  if denominator == 0:
    return 0
  else:
    return numerator/denominator

In [24]:
predicted_rating = predict(0,1)
predicted_rating

tensor(3.)